In [6]:
import numpy as np
from math import pi
from copy import deepcopy
from qiskit import *
import matplotlib.pyplot as plt

In [7]:
def expectationValue(circuit, q, c):
    measurement = deepcopy(circuit)
    measurement.measure(q, c)
    shots = 10000
    job = execute(measurement, backend = Aer.get_backend('qasm_simulator'), shots = shots)
    counts = job.result().get_counts(measurement)
    
    #If elements exist add them to the expectationValue counts
    expectationValue = (1 * counts.get('00', 0) + 1 * counts.get('11',0) + -1 * counts.get('10', 0) + -1 * counts.get('01',0))/shots
    #return the expectation value
    return expectationValue

In [8]:
def densityMatrixComponents(circuit,q,c):
    eValues = {}
    circuits = []
    for i in range(15):
        circuits.append(deepcopy(circuit))
    
#Measure xyz components of first qubit
    
    
    #Measures X
    circuits[0].ry(pi/2,1)
    expectValueXI = expectationValue(circuits[0], q[0], c[0])
    eValues['XI'] = expectValueXI
    
    #Measure Y
    circuits[1].rx(pi/2,1)
    expectValueYI = expectationValue(circuits[1], q[1], c[1])
    eValues['YI'] = expectValueYI
    
    #Measure Z
    expectValueZI = expectationValue(circuits[2], q[1], c[1])
    eValues['ZI'] = expectValueZI
    
#Measure xyz components of second qubit

    #Measure X
    circuits[3].ry(pi/2,0)
    expectValueIX = expectationValue(circuits[3], q[0], c[0])
    eValues['IX'] = expectValueIX
    
    #Measure Y
    circuits[4].rx(pi/2,0)
    expectValueIY = expectationValue(circuits[4], q[0], c[0])
    eValues['IY'] = expectValueIY
    
    #Measure Z
    expectValueIZ = expectationValue(circuits[5], q[1], c[1])
    eValues['IZ'] = expectValueIZ
    
#Measure qubit 1 along X and vary qubit 2

    circuits[6].ry(pi/2,1)
    circuits[6].ry(pi/2,0)
    expectValueXX = expectationValue(circuits[6],q,c)
    eValues['XX'] = expectValueXX
    
    circuits[7].ry(pi/2,1)
    circuits[7].rx(pi/2,0)
    expectValueXY = expectationValue(circuits[7],q,c)
    eValues['XY'] = expectValueXY
    
    circuits[8].ry(pi/2,1)
    expectValueXZ = expectationValue(circuits[8],q,c)
    eValues['XZ'] = expectValueXZ
    
#Measure qubit 1 along Y vary qubit 2

    circuits[9].rx(pi/2,1)
    circuits[9].ry(pi/2,0)
    expectValueYX = expectationValue(circuits[9],q,c)
    eValues['YX'] = expectValueYX
    
    circuits[10].rx(pi/2,1)
    circuits[10].rx(pi/2,0)
    expectValueYY = expectationValue(circuits[10],q,c)
    eValues['YY'] = expectValueYY
    
    circuits[11].rx(pi/2,1)
    expectValueYZ = expectationValue(circuits[11],q,c)
    eValues['YZ'] = expectValueYZ
    
#Measure qubit 1 along Z vary qubit 2

    circuits[12].ry(pi/2,0)
    expectValueZX = expectationValue(circuits[12],q,c)
    eValues['ZX'] = expectValueZX
    
    circuits[13].rx(pi/2,0)
    expectValueZY = expectationValue(circuits[13],q,c)
    eValues['ZY'] = expectValueZY
    
    expectValueZZ = expectationValue(circuits[14],q,c)
    eValues['ZZ'] = expectValueZZ
    
    for i in eValues:
        eValues[i] = round(eValues[i])
    
    return eValues

In [9]:
q = QuantumRegister(2)
c = ClassicalRegister(2)
circ0 = QuantumCircuit(q, c)
circ0.x(0)
circ0.ry(np.pi/2, 1)
circ0.cx(q[1], q[0])

densityMatrixComponents(circ0, q, c)

{'XI': 0,
 'YI': 0,
 'ZI': 0,
 'IX': 0,
 'IY': 0,
 'IZ': 0,
 'XX': 1,
 'XY': 0,
 'XZ': 0,
 'YX': 0,
 'YY': 1,
 'YZ': 0,
 'ZX': 0,
 'ZY': 0,
 'ZZ': -1}

In [10]:
def getUnitary(circuit):
    circ = deepcopy(circuit)
    job = execute(circ, backend = BasicAer.get_backend('unitary_simulator'))
    M = job.result().get_unitary(circ,decimals=3)
    return M

q = QuantumRegister(2)
c = ClassicalRegister(2)
circ = QuantumCircuit(q, c)
circ.x(0)
circ.x(1)

getUnitary(circ)

def getDensityMatrix(circuit, q, c):
    components = densityMatrixComponents(circuit,q,c)
    #print(components)
    q = QuantumRegister(2)
    c = ClassicalRegister(2)
    circ = QuantumCircuit(q,c)
    
    #XI
    circXI = deepcopy(circ)
    circXI.x(1)
    XI = getUnitary(circXI)*components.get('XI')
    
    #YI
    circYI = deepcopy(circ)
    circYI.y(1)
    YI = getUnitary(circYI)*components.get('YI')
    
    #ZI
    circZI = deepcopy(circ)
    circZI.z(1)
    ZI = getUnitary(circZI)*components.get('ZI')
    
    #IX
    circIX = deepcopy(circ)
    circIX.x(0)
    IX = getUnitary(circIX)*components.get('IX')
    
    #IY
    circIY = deepcopy(circ)
    circIY.y(0)
    IY = getUnitary(circIY)*components.get('IY')
    
    #IZ
    circIZ = deepcopy(circ)
    circIZ.z(0)
    IZ = getUnitary(circIZ)*components.get('IZ')
    
    #XX
    circXX = deepcopy(circ)
    circXX.x(1)
    circXX.x(0)
    XX = getUnitary(circXX)*components.get('XX')
    
    #XY
    circXY = deepcopy(circ)
    circXY.x(1)
    circXY.y(0)
    XY = getUnitary(circXY)*components.get('XY')
    
    #XZ
    circXZ = deepcopy(circ)
    circXZ.x(1)
    circXZ.z(0)
    XZ = getUnitary(circXZ)*components.get('XZ')
    
    #YX
    circYX = deepcopy(circ)
    circYX.y(1)
    circYX.x(0)
    YX = getUnitary(circYX)*components.get('YX')
    
    #YY
    circYY = deepcopy(circ)
    circYY.y(1)
    circYY.y(0)
    YY = getUnitary(circYY)*components.get('YY')
    
    #YZ
    circYZ = deepcopy(circ)
    circYZ.y(1)
    circYZ.z(0)
    YZ = getUnitary(circYZ)*components.get('YZ')
    
    #ZX
    circZX = deepcopy(circ)
    circZX.z(1)
    circZX.x(0)
    ZX = getUnitary(circZX)*components.get('ZX')
    
    #ZY
    circZY = deepcopy(circ)
    circZY.z(1)
    circZY.y(0)
    ZY = getUnitary(circZY)*components.get('ZY')
    
    #ZZ
    circZZ = deepcopy(circ)
    circZZ.z(1)
    circZZ.z(0)
    ZZ = getUnitary(circZZ)*components.get('ZZ')
    
    circII = deepcopy(circ)
    II = getUnitary(circII)
    
    densityMatrix = (1/4)*(IX + IY + IZ + XI + YI + ZI + XX + XY + XZ + YX + YY + YZ + ZX + ZY + ZZ + II)
    return densityMatrix
    
q0 = QuantumRegister(2)
c0 = ClassicalRegister(2)
T1 = QuantumCircuit(q0, c0)
T1.x(0)
T1.ry(pi/2, 1)
T1.cx(q0[1], q0[0])
    
getDensityMatrix(T1, q0, c0)
x

array([[0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j, 0.5+0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j]])